In [1]:
import pandas as pd
pd.set_option('display.max_rows', 100, 'display.max_columns', 200)

In [2]:
zip_income=pd.read_excel('MedianZIP-3.xlsx')

In [3]:
zip_group={}
for i in range(10,1000):
    mean_value = zip_income[(zip_income['Zip']>=(i*100)) & (zip_income['Zip']<((i+1)*100))]['Median'].mean()
    if mean_value == mean_value:
        zip_group[i]= mean_value

In [4]:
def clean_data(data):
    data.dropna(axis=0, how='all', inplace=True)
    data['loan_status'] = data['loan_status'].replace(r'Does not meet the credit policy. Status:','',regex=True)
    data['term']=data['term'].replace([r'36*',r'60*'],[36,60],regex=True)
    data['int_rate'] = data['int_rate'].replace('%','',regex=True).astype('float')/100
    
    data['sub_grade'] = data['sub_grade'].replace(r'[A-Z]','',regex=True).astype('int')
    
    data['emp_length']=data['emp_length'].replace([r'<1*',r'10\+*','n/a'],[0.5,11,0],regex=True)
    data['emp_length'] = data['emp_length'].replace(r'year.*','',regex=True).astype('float')
    
    data.drop(data[data.loan_status == 'Current'].index, inplace=True)
    data.loc[data['loan_status'] != 'Fully Paid', 'loan_status'] = 0
    data.loc[data['loan_status'] == 'Fully Paid', 'loan_status'] = 1
    data.loc[(data['home_ownership'] == 'ANY') | (data['home_ownership'] == 'NONE'), 'home_ownership'] = 'OTHER'
    
    data['zip_income'] = -1
    zip_mean= zip_income['Median'].mean()
    for index,row in data.iterrows():
        value = zip_group.get(int(row['zip_code'][:3]), zip_mean)
        data.set_value(index,'zip_income',value) 
    data.drop(['zip_code'], axis=1, inplace= True)

In [5]:
cols_=['loan_amnt','term','int_rate','installment','grade','sub_grade','emp_length','home_ownership','annual_inc',
       'verification_status','zip_code','purpose','loan_status']
data_3a=pd.read_csv('LoanStats3a.csv',skiprows=[0],usecols=cols_)
clean_data(data_3a)
data_3b=pd.read_csv('LoanStats3b.csv',skiprows=[0],usecols=cols_)
clean_data(data_3b)
data_3c=pd.read_csv('LoanStats3c.csv',skiprows=[0],usecols=cols_)
clean_data(data_3c)

In [6]:
data_3a.groupby(['loan_status']).size()

loan_status
0     6431
1    36104
dtype: int64

In [7]:
data_3b.groupby(['loan_status']).size()

loan_status
0     29290
1    148092
dtype: int64

In [8]:
data_3c.groupby(['loan_status']).size()

loan_status
0     38873
1    134710
dtype: int64

In [9]:
data= pd.concat([data_3a,data_3b,data_3c],ignore_index=True)
data.drop_duplicates(inplace=True)
data=data.reset_index(drop=True)

In [10]:
data.to_pickle('data.p')